In [1]:
from IPython.display import display,HTML
def dhtml(str):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family=Akronim&effect=3d';      
    </style><h1 class='font-effect-3d' 
    style='font-family:Akronim; color:#33ffcc;'>
    %s</h1>"""%str))

Reading classics [Deep Learning Models](https://nbviewer.jupyter.org/github/rasbt/deeplearning-models/blob/master/pytorch_ipynb/rnn/rnn_lstm_packed_imdb.ipynb)

In [2]:
dhtml('Code Modules, Functions, & Classes')

In [3]:
import numpy as np,pylab as pl
import torch,random,spacy,pandas as pd
import torch.nn as tnn
import torch.nn.functional as tnnf
from torchtext import data as ttdata
from torchtext import datasets as ttds
torch.backends.cudnn.deterministic=True
dev=torch.device('cuda' \
if torch.cuda.is_available() else 'cpu')
nlp=spacy.load('en')

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,\
MaxPooling1D,Dense,LSTM,Embedding
from tensorflow.keras.preprocessing import \
sequence as ksequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import \
ModelCheckpoint,ReduceLROnPlateau

In [5]:
def bin_accuracy(model,data_loader):
    model.eval()
    correct_pred,num_examples=0,0
    with torch.no_grad():
        for batch_ids,batch_data in enumerate(data_loader):
            text,text_lengths=batch_data.text
            logits=model(text,text_lengths)
            predicted_labels=(torch.sigmoid(logits)>.5).long()
            num_examples+=batch_data.label.size(0)
            correct_pred+=(predicted_labels==\
                           batch_data.label.long()).sum()
        return correct_pred.float()/num_examples*100
def predict_sentiment(model,sentence):
    model.eval()
    tokenized=[tok.text for tok in nlp.tokenizer(sentence)]
    indexed=[ttext.vocab.stoi[t] for t in tokenized]
    length_tensor=torch.LongTensor([len(indexed)])
    tensor=torch.LongTensor(indexed).to(dev)
    tensor=tensor.unsqueeze(1)
    prediction=torch.sigmoid(model(tensor,length_tensor))
    return prediction.item() 

In [6]:
dhtml('Data')

In [7]:
random_state=random.seed(12)
ids='train valid test'.split()
ttext=ttdata.Field(tokenize='spacy',
                   include_lengths=True)
tlabel=ttdata.LabelField(dtype=torch.float)
train,test=ttds.IMDB.splits(ttext,tlabel)
train,valid=train\
.split(random_state=random_state,split_ratio=.8)
pd.DataFrame([len(el) for el in [train,valid,test]],
    index=ids,columns=['length']).T

,train,valid,test
length,20000,5000,25000


In [8]:
random_seed=23
torch.manual_seed(random_seed)
vocab_size=20000; batch_size=128
ttext.build_vocab(train,max_size=vocab_size)
tlabel.build_vocab(train)
num_clases=tlabel.vocab
input_dim=len(ttext.vocab); output_dim=1
train_loader,valid_loader,test_loader=ttdata.BucketIterator\
.splits((train,valid,test),\
batch_size=batch_size,sort_within_batch=True,device=dev)
dataloaders={ids[0]:train_loader,
             ids[1]:valid_loader,
             ids[2]:test_loader}
for i in range(3):
    print(ids[i])
    for batch in dataloaders[ids[i]]:
        print(f'text matrix: {batch.text[0].size()}')
        print(f'label vector: {batch.label.size()}')
        break

train
text matrix: torch.Size([154, 128])
label vector: torch.Size([128])
valid
text matrix: torch.Size([57, 128])
label vector: torch.Size([128])
test
text matrix: torch.Size([42, 128])
label vector: torch.Size([128])


In [9]:
# 25,000 movies reviews from IMDB, 
# labeled by sentiment (positive/negative)
num_words,max_length,embedding_vector_len=\
10000,1000,32
(x_train,y_train),(x_test,y_test)=\
imdb.load_data(path="imdb_full.pkl",num_words=num_words,
               skip_top=0,maxlen=max_length,seed=113,
               start_char=1,oov_char=2,index_from=3)
x_test,x_valid,y_test,y_valid=\
train_test_split(x_test,y_test,
                 test_size=.5,random_state=1)

In [10]:
print(x_train.shape,x_valid.shape,x_test.shape)
print('Label: ',y_train[1]) 
print('Sequence of word indices: \n',x_train[1])
px_train=ksequence\
.pad_sequences(x_train,maxlen=max_length)
px_valid=ksequence\
.pad_sequences(x_valid,maxlen=max_length)
px_test=ksequence\
.pad_sequences(x_test,maxlen=max_length)

(25000,) (12410,) (12409,)
Label:  0
Sequence of word indices: 
 [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


In [11]:
dhtml('Basic RNNs')

In [12]:
class RNN(tnn.Module):
    def __init__(self,input_dim,embedding_dim,
                 hidden_dim,output_dim):      
        super().__init__()    
        self.embedding=tnn.Embedding(input_dim,embedding_dim)
        self.rnn=tnn.LSTM(embedding_dim,hidden_dim)
        self.fc=tnn.Linear(hidden_dim,output_dim)    
    def forward(self,text,text_length):
        embedded=self.embedding(text)
        packed=tnn.utils.rnn\
        .pack_padded_sequence(embedded,text_length)
        packed_output,(hidden,cell)=self.rnn(packed)
        return self.fc(hidden.squeeze(0)).view(-1)

In [13]:
def kmodel():
    model=Sequential()
    model.add(Embedding(num_words,embedding_vector_len,
                        input_length=max_length))
    model.add(Conv1D(filters=32,kernel_size=3,
                     padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2))   
    model.add(LSTM(32))    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='nadam',metrics=['accuracy'])    
    return model

In [14]:
dhtml('Training')

In [15]:
random_seed=34; learning_rate=1e-4
torch.manual_seed(random_seed)
embedding_dim=128; hidden_dim=256
model=RNN(input_dim,embedding_dim,
          hidden_dim,output_dim)
model=model.to(dev)
optimizer=torch.optim\
.Adam(model.parameters(),lr=learning_rate)

In [16]:
epochs=15
for epoch in range(epochs):
    model.train()
    for batch_ids,batch_data in enumerate(train_loader):
        text,text_lengths=batch_data.text
        logits=model(text,text_lengths)
        cost=tnnf.binary_cross_entropy_with_logits(
            logits,batch_data.label)
        optimizer.zero_grad()
        cost.backward(); optimizer.step()
        if not batch_ids%100:
            print (f'Epoch: {epoch+1:03d}/{epochs:03d} | '
                   f'Batch: {batch_ids:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.6f}')         
    with torch.set_grad_enabled(False):
        print(f'training acc: '
              f'{bin_accuracy(model,train_loader):.2f}%'
              f'\nvalid acc: '
              f'{bin_accuracy(model,valid_loader):.2f}%')
model.eval()
print(f'test acc: {bin_accuracy(model,test_loader):.2f}%')

Epoch: 001/015 | Batch: 000/157 | Cost: 0.692748
Epoch: 001/015 | Batch: 100/157 | Cost: 0.691674
training acc: 58.66%
valid acc: 56.36%
Epoch: 002/015 | Batch: 000/157 | Cost: 0.680379
Epoch: 002/015 | Batch: 100/157 | Cost: 0.631539
training acc: 66.10%
valid acc: 64.46%
Epoch: 003/015 | Batch: 000/157 | Cost: 0.668875
Epoch: 003/015 | Batch: 100/157 | Cost: 0.527250
training acc: 75.58%
valid acc: 75.14%
Epoch: 004/015 | Batch: 000/157 | Cost: 0.436630
Epoch: 004/015 | Batch: 100/157 | Cost: 0.572089
training acc: 78.36%
valid acc: 76.30%
Epoch: 005/015 | Batch: 000/157 | Cost: 0.456969
Epoch: 005/015 | Batch: 100/157 | Cost: 0.473156
training acc: 81.03%
valid acc: 79.18%
Epoch: 006/015 | Batch: 000/157 | Cost: 0.489449
Epoch: 006/015 | Batch: 100/157 | Cost: 0.434458
training acc: 81.93%
valid acc: 80.32%
Epoch: 007/015 | Batch: 000/157 | Cost: 0.423922
Epoch: 007/015 | Batch: 100/157 | Cost: 0.462577
training acc: 83.84%
valid acc: 81.80%
Epoch: 008/015 | Batch: 000/157 | Cost: 0

In [17]:
kmodel=kmodel()
fw='weights.best.hdf5'
checkpointer=\
ModelCheckpoint(filepath=fw,verbose=2,
                save_best_only=True)
lr_reduction=\
ReduceLROnPlateau(monitor='val_loss',patience=10,
                  verbose=2,factor=.5)
history=kmodel.fit(px_train,y_train,epochs=5,batch_size=128,
                   validation_data=(px_valid,y_valid),
                   callbacks=[checkpointer,lr_reduction])
kmodel.load_weights(fw)
kmodel.evaluate(px_test,y_test)

Epoch 1/5
196/196 [==============================] - ETA: 0s - loss: 0.4541 - accuracy: 0.7800
Epoch 00001: val_loss improved from inf to 0.30673, saving model to weights.best.hdf5
196/196 [==============================] - 23s 119ms/step - loss: 0.4541 - accuracy: 0.7800 - val_loss: 0.3067 - val_accuracy: 0.8733 - lr: 0.0010
Epoch 2/5
196/196 [==============================] - ETA: 0s - loss: 0.2373 - accuracy: 0.9108
Epoch 00002: val_loss improved from 0.30673 to 0.29348, saving model to weights.best.hdf5
196/196 [==============================] - 22s 114ms/step - loss: 0.2373 - accuracy: 0.9108 - val_loss: 0.2935 - val_accuracy: 0.8820 - lr: 0.0010
Epoch 3/5
196/196 [==============================] - ETA: 0s - loss: 0.1792 - accuracy: 0.9363
Epoch 00003: val_loss did not improve from 0.29348
196/196 [==============================] - 22s 112ms/step - loss: 0.1792 - accuracy: 0.9363 - val_loss: 0.3140 - val_accuracy: 0.8781 - lr: 0.0010
Epoch 4/5
196/196 [============================

[0.2916161119937897, 0.8850834369659424]

In [18]:
dhtml('Predictions')

In [19]:
print('probability of being positive:')
sent='I really love this movie. The actor team here is so great!'
predict_sentiment(model,sent)

probability of being positive:


0.8204745650291443

In [20]:
word2index=imdb.get_word_index()
sent=sent.lower().replace('.','').replace('!','')
sent_index=[]
for word in sent.split():
     sent_index.append(word2index[word])
sent_index=ksequence.pad_sequences([sent_index],
                                   maxlen=max_length)
kmodel.predict(sent_index)

array([[0.5782955]], dtype=float32)